In [313]:
import pandas as pd
import os
import re
import numpy as np
import pprint as pp

In [223]:
os.getcwd()

'D:\\Project- Electro\\scripts'

In [224]:
path_to_file = os.path.join('..','Summary of free interaction - 1-2023','My Rat Experiments-08.01.2023_copy.xlsx')
df = pd.read_excel(path_to_file,'data')

In [225]:
area_names = [c for c in df.columns if 'Area' in c]

In [226]:
def get_rat_num(text):
    pattern = r'(?i)rat\s*(\d+(?:\.\d+)?)'
    matches = re.findall(pattern, text)
    numbers = [int(match) for match in matches]
    return numbers[0]

def get_day_num(text):
    pattern = r'(?i)day\s*(\d+(?:\.\d+)?)'
    matches = re.findall(pattern, text)
    numbers = [int(match) for match in matches]
    return numbers[0]
def get_if_sniffing(x):
    return 'sniff' in x.lower()

def get_prob_name(text):
    pattern = r'(?i)probe\s*(\d+(?:\.\d+)?)'
    matches = re.findall(pattern, text)
    numbers = [int(match) for match in matches]
    return numbers[0]
    

def get_affiliative(x):
    if 'affiliative' in x.lower():
        return 'affiliative'
    elif 'aversive' in x.lower():
        return 'aversive'
    else:
        return -1

def get_brain_areas(df_in):
    return [c for c in df_in.index if 'Area' in c and not pd.isnull(df_in[c])]

def recode_area_names(df_in):
    
    b_areas = df_in[[c for c in area_names]]
    vals = [not pd.isnull(v) for v in b_areas.values]
    df_b = pd.DataFrame(zip(vals)).transpose()
    df_b.columns = area_names
    return df_b

In [227]:
def pars_rat_id(df_in):
    text = df_in['Rat and session number']
    rat_num = get_rat_num(text)
    day_num = get_day_num(text)
    prob_num = get_prob_name(text)
    sniffing = get_if_sniffing(text)
    
    affiliative = get_affiliative(df_in['Brain state'])
    brain_areas_list = get_brain_areas(df_in)
    brain_areas = recode_area_names(df_in)
    
    d_out = pd.DataFrame(zip([rat_num,day_num,prob_num,sniffing,affiliative, brain_areas_list])).transpose()
    columns=['rat_num','day_num','prob_num','sniffing','affiliative','brain_areas_list']
    d_out.columns = columns
    d_out = pd.concat([d_out,brain_areas], axis=1)
    return d_out
    

In [228]:
d_out = [pars_rat_id(df.iloc[i]) for i in range(len(df))]
d_out = pd.concat(d_out)
d_out.reset_index(drop=True)

,rat_num,day_num,prob_num,sniffing,affiliative,brain_areas_list,Area 111 MeA,Area 2 MePV,Area 13 CeA,Area 112 BMA,Area 14 AA/ACO/VEN,Area 16 EA,Area 12 STIA,Area 15 VP,Area 113 AHiA
0,1,1,4,False,affiliative,"[Area 13 CeA, Area 12 STIA]",False,False,True,False,False,False,True,False,False
1,1,2,4,False,affiliative,"[Area 13 CeA, Area 12 STIA]",False,False,True,False,False,False,True,False,False
2,2,1,1,True,affiliative,"[Area 111 MeA, Area 16 EA]",True,False,False,False,False,True,False,False,False
3,2,2,1,True,affiliative,"[Area 111 MeA, Area 16 EA]",True,False,False,False,False,True,False,False,False
4,2,3,1,True,affiliative,"[Area 111 MeA, Area 16 EA]",True,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,18,2,19,False,aversive,"[Area 2 MePV, Area 112 BMA, Area 113 AHiA]",False,True,False,True,False,False,False,False,True
59,18,3,19,False,aversive,"[Area 2 MePV, Area 112 BMA, Area 113 AHiA]",False,True,False,True,False,False,False,False,True
60,19,1,18,False,aversive,"[Area 111 MeA, Area 112 BMA, Area 14 AA/ACO/VEN]",True,False,False,True,True,False,False,False,False
61,19,3,20,False,aversive,"[Area 111 MeA, Area 112 BMA, Area 113 AHiA]",True,False,False,True,False,False,False,False,True


## Brain area examined per rat

In [326]:
def get_num_of_sessions(area, verbose=False):
    affiliation = 'affiliative'

    df = d_out
    affiliative_rats_for_area = df.loc[(df[area]==True) & (df['affiliative']==affiliation)]['rat_num']
    num_rats_affiliative = len(set(affiliative_rats_for_area))
    num_sessions_affiliative = len(affiliative_rats_for_area)

    affiliation = 'aversive'
    aversive_rats_for_area = df.loc[(df[area]==True) & (df['affiliative']==affiliation)]['rat_num']
    num_rats_aversive = len(set(aversive_rats_for_area))
    num_sessions_aversive = len(aversive_rats_for_area)

    if verbose:
        print(f'{area} was messured in {num_rats_aversive} aversive rats with {num_sessions} sessions and {num_rats_affiliative} affiliative rats with {num_sessions_affiliative} sessions')
    
    df_out = pd.DataFrame(zip([area, num_rats_affiliative, num_sessions_affiliative, num_rats_aversive, num_sessions_aversive])).transpose()
    df_out.columns = ['area','affiliative rats','affiliative_total','aversive_rats','aversive_total']
    return df_out

In [301]:
df_sum_areas = pd.concat([get_num_of_sessions(area) for area in area_names])
df_sum_areas = df_sum_areas.set_index('area')
df_sum_areas


,affiliative rats,affiliative_total,aversive_rats,aversive_total
area,,,,
Area 111 MeA,14,29,10,24
Area 2 MePV,6,14,3,7
Area 13 CeA,7,9,6,13
Area 112 BMA,7,19,4,10
Area 14 AA/ACO/VEN,10,14,7,15
Area 16 EA,5,8,4,11
Area 12 STIA,6,7,5,12
Area 15 VP,3,6,1,2
Area 113 AHiA,2,5,2,4


We can see that areas 15 VP and 113 AHiA have fewer session than 5 and will be excluded

In [337]:
df_sum_areas = pd.concat([get_num_of_sessions(area) for area in area_names])
df_sum_areas = df_sum_areas.set_index('area')
df_sum_areas

selected_ind = df_sum_areas[['affiliative_total','aversive_total']].values
selected_ind = np.all(selected_ind>=5, axis=1)
df_sum_areas = df_sum_areas.loc[selected_ind]
area_names_filter = [a for (a, s) in zip(area_names, selected_ind) if s]
df_sum_areas

,affiliative rats,affiliative_total,aversive_rats,aversive_total
area,,,,
Area 111 MeA,14,29,10,24
Area 2 MePV,6,14,3,7
Area 13 CeA,7,9,6,13
Area 112 BMA,7,19,4,10
Area 14 AA/ACO/VEN,10,14,7,15
Area 16 EA,5,8,4,11
Area 12 STIA,6,7,5,12


affiliative and aversive rats

In [271]:
affiliative_rats = d_out.loc[(d_out['affiliative']=='affiliative')]['rat_num'].unique()
aversive_rats = d_out.loc[(d_out['affiliative']=='aversive')]['rat_num'].unique()

print(f'Rats in aversive sessions: {aversive_rats}')
print(f'Rats in affiliative sessions: {affiliative_rats}')
rats_in_both_groups = set(aversive_rats).intersection(affiliative_rats)
print(f'Rats in both aversive and affiliative sessions: {rats_in_both_groups}')



Rats in aversive sessions: [3 4 9 10 12 15 16 17 18 19 23]
Rats in affiliative sessions: [1 2 3 4 5 8 9 12 14 15 16 19 20 21 22 23 26]
Rats in both aversive and affiliative sessions: {3, 4, 9, 12, 15, 16, 19, 23}


## Find pairs of areas common in both affiliative and aversive sessions

In [316]:
import itertools

area_names_filter
all_pairs = itertools.combinations(area_names_filter, 2)
all_pairs = list(all_pairs)
pp.pprint(f'There are {len(all_pairs)} pairs:')
pp.pprint(all_pairs)


'There are 21 pairs:'
[('Area 111 MeA', 'Area 2 MePV'),
 ('Area 111 MeA', 'Area 13 CeA'),
 ('Area 111 MeA', 'Area 112 BMA'),
 ('Area 111 MeA', 'Area 14 AA/ACO/VEN'),
 ('Area 111 MeA', 'Area 16 EA'),
 ('Area 111 MeA', 'Area 12 STIA'),
 ('Area 2 MePV', 'Area 13 CeA'),
 ('Area 2 MePV', 'Area 112 BMA'),
 ('Area 2 MePV', 'Area 14 AA/ACO/VEN'),
 ('Area 2 MePV', 'Area 16 EA'),
 ('Area 2 MePV', 'Area 12 STIA'),
 ('Area 13 CeA', 'Area 112 BMA'),
 ('Area 13 CeA', 'Area 14 AA/ACO/VEN'),
 ('Area 13 CeA', 'Area 16 EA'),
 ('Area 13 CeA', 'Area 12 STIA'),
 ('Area 112 BMA', 'Area 14 AA/ACO/VEN'),
 ('Area 112 BMA', 'Area 16 EA'),
 ('Area 112 BMA', 'Area 12 STIA'),
 ('Area 14 AA/ACO/VEN', 'Area 16 EA'),
 ('Area 14 AA/ACO/VEN', 'Area 12 STIA'),
 ('Area 16 EA', 'Area 12 STIA')]


In [320]:
df_pairs = d_out.copy()
for (p1, p2) in all_pairs:
    df_pairs[(p1,p2)] = np.all(df_pairs[[p1,p2]].values, axis=1)

In [335]:
# Count the number of pairs messured in for both session types
def get_num_of_sessions_2(df, pair_or_areas, verbose=False):
    affiliation = 'affiliative'

    affiliative_rats_for_area = df.loc[(df[pair_or_areas]==True) & (df['affiliative']==affiliation)]['rat_num']
    num_rats_affiliative = len(set(affiliative_rats_for_area))
    num_sessions_affiliative = len(affiliative_rats_for_area)

    affiliation = 'aversive'
    aversive_rats_for_area = df.loc[(df[pair_or_areas]==True) & (df['affiliative']==affiliation)]['rat_num']
    num_rats_aversive = len(set(aversive_rats_for_area))
    num_sessions_aversive = len(aversive_rats_for_area)

    if verbose:
        print(f'{pair_or_areas} was messured in {num_rats_aversive} aversive rats with {num_sessions} sessions and {num_rats_affiliative} affiliative rats with {num_sessions_affiliative} sessions')
    
    df_out = pd.DataFrame(zip([pair_or_areas, num_rats_affiliative, num_sessions_affiliative, num_rats_aversive, num_sessions_aversive])).transpose()
    df_out.columns = ['pair_or_areas','affiliative rats','affiliative_total','aversive_rats','aversive_total']
    return df_out


In [344]:
df_sum_area_pairs = pd.concat([get_num_of_sessions_2(df_pairs, area) for area in all_pairs])
df_sum_area_pairs = df_sum_area_pairs.set_index('pair_or_areas')
df_sum_area_pairs



,affiliative rats,affiliative_total,aversive_rats,aversive_total
pair_or_areas,,,,
"(Area 111 MeA, Area 2 MePV)",6,12,2,4
"(Area 111 MeA, Area 13 CeA)",5,6,6,12
"(Area 111 MeA, Area 112 BMA)",6,16,3,7
"(Area 111 MeA, Area 14 AA/ACO/VEN)",8,12,7,14
"(Area 111 MeA, Area 16 EA)",5,8,4,11
"(Area 111 MeA, Area 12 STIA)",5,5,5,11
"(Area 2 MePV, Area 13 CeA)",1,1,1,1
"(Area 2 MePV, Area 112 BMA)",3,10,2,6
"(Area 2 MePV, Area 14 AA/ACO/VEN)",4,6,2,4


In [345]:
# We can see that some pairs were not measured in at least 5 session in both aversive and affiliative sessions

In [346]:
selected_ind = df_sum_area_pairs[['affiliative_total','aversive_total']].values
selected_ind = np.all(selected_ind>=5, axis=1)
df_sum_area_pairs = df_sum_area_pairs.loc[selected_ind]
area_names_filter = [a for (a, s) in zip(area_names, selected_ind) if s]
df_sum_area_pairs

,affiliative rats,affiliative_total,aversive_rats,aversive_total
pair_or_areas,,,,
"(Area 111 MeA, Area 13 CeA)",5,6,6,12
"(Area 111 MeA, Area 112 BMA)",6,16,3,7
"(Area 111 MeA, Area 14 AA/ACO/VEN)",8,12,7,14
"(Area 111 MeA, Area 16 EA)",5,8,4,11
"(Area 111 MeA, Area 12 STIA)",5,5,5,11
"(Area 2 MePV, Area 112 BMA)",3,10,2,6
"(Area 13 CeA, Area 12 STIA)",4,5,4,10
"(Area 112 BMA, Area 14 AA/ACO/VEN)",4,7,3,6
